<a href="https://colab.research.google.com/github/manpritdueby/data-cleaning/blob/main/NL2sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl duckdb g4f


In [ ]:
import pandas as pd
import os

# For Colab
try:
    from google.colab import files
    uploaded = files.upload()
except:
    uploaded = {file: file for file in os.listdir() if file.endswith(('.csv', '.xlsx', '.xls', '.json'))}

def load_dataset(uploaded):
    for filename in uploaded:
        ext = filename.split('.')[-1].lower()
        if ext == 'csv':
            df = pd.read_csv(filename)
        elif ext in ['xls', 'xlsx']:
            df = pd.read_excel(filename)
        elif ext == 'json':
            df = pd.read_json(filename)
        else:
            raise ValueError("Unsupported file format")
        df.to_csv("data.csv", index=False)  # Save to CSV for DuckDB
        return df

df = load_dataset(uploaded)
print("✅ Dataset loaded successfully!")
print(df.head())


Saving phone_usage_india.xlsx to phone_usage_india (1).xlsx
✅ Dataset loaded successfully!
   User ID  Age  Gender   Location Phone Brand       OS  \
0        1   53    Male     Mumbai        Vivo  Android   
1        2   60   Other      Delhi      Realme      iOS   
2        3   37  Female  Ahmedabad       Nokia  Android   
3        4   32    Male       Pune     Samsung  Android   
4        5   16    Male     Mumbai      Xiaomi      iOS   

   Screen Time (hrs/day)  Data Usage (GB/month)  Calls Duration (mins/day)  \
0                    3.7                   23.9                       37.9   
1                    9.2                   28.1                       13.7   
2                    4.5                   12.3                       66.8   
3                   11.0                   25.6                      156.2   
4                    2.2                    2.5                      236.2   

   Number of Apps Installed  Social Media Time (hrs/day)  \
0                       1

In [ ]:
import g4f

def generate_sql_from_nl(question, df):
    schema = ", ".join([f"{col} ({dtype})" for col, dtype in zip(df.columns, df.dtypes)])
    prompt = f"""
You are an AI that converts English to SQL.

Use the table: read_csv_auto('data.csv')

Dataset Schema:
{schema}

User Question: {question}

Write a complete SQL query:
"""
    response = g4f.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    raw_sql = response.strip()
    # Remove markdown formatting if present
    if raw_sql.startswith("```"):
        raw_sql = raw_sql.strip("```sql").strip("```").strip()
    return raw_sql


In [ ]:
import duckdb

def run_query_on_df(sql_query):
    try:
        result = duckdb.query(sql_query).to_df()
        return result
    except Exception as e:
        print("❌ SQL Error:", e)
        return None


In [ ]:
print("\n💬 Ask questions about your dataset (type 'exit' to quit)\n")

while True:
    question = input("🗨️ Your Question: ")
    if question.lower() == 'exit':
        print("👋 Exiting.")
        break

    print("\n⏳ Generating SQL query...")
    sql_query = generate_sql_from_nl(question, df)

    print("\n🧠 Generated SQL:")
    print("────────────────────────────")
    print(sql_query)
    print("────────────────────────────")

    print("\n🔍 Running SQL on dataset...")
    result = run_query_on_df(sql_query)

    if result is not None:
        print("\n✅ Result:")
        print(result)
    else:
        print("❌ Query failed. Try rephrasing your question.")



💬 Ask questions about your dataset (type 'exit' to quit)


⏳ Generating SQL query...

🧠 Generated SQL:
────────────────────────────
SELECT COUNT(DISTINCT "Phone Brand") AS total_phone_brands
FROM read_csv_auto('data.csv');
────────────────────────────

🔍 Running SQL on dataset...

✅ Result:
   total_phone_brands
0                  10

⏳ Generating SQL query...

🧠 Generated SQL:
────────────────────────────
Here is the SQL query that retrieves the total number of distinct phone brands where the primary use is "education":

```sql
SELECT COUNT(DISTINCT "Phone Brand") AS total_phone_brands
FROM read_csv_auto('data.csv')
WHERE "Primary Use" = 'education';
```
────────────────────────────

🔍 Running SQL on dataset...
❌ SQL Error: Parser Error: syntax error at or near "Here"
❌ Query failed. Try rephrasing your question.

⏳ Generating SQL query...

🧠 Generated SQL:
────────────────────────────
SELECT COUNT(*) AS total_male_apple_users
FROM read_csv_auto('data.csv')
WHERE Gender = 'Male' 
  A